# Day 5: Central Limit Theorem & Confidence Intervals
## Week 2: Statistics & Probability for Finance

---

**Learning Objectives:**
- Understand CLT and why it matters for finance
- Construct confidence intervals for returns and Sharpe ratios
- Apply bootstrap methods for robust estimation
- Quantify estimation uncertainty

In [ ]:
# Day 5 Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')

# Load market data
df = pd.read_csv('../datasets/raw_data/combined_adjusted_close.csv', 
                 index_col='Date', parse_dates=True)
prices = df[['AAPL', 'MSFT', 'SPY', 'JPM']].dropna()
returns = prices.pct_change().dropna()

print("=" * 60)
print("CLT & CONFIDENCE INTERVALS - DAY 5")
print("=" * 60)

## 1. Central Limit Theorem Demonstration

**CLT States:** The distribution of sample means approaches Normal as n → ∞, regardless of the underlying distribution.

$$\bar{X} \xrightarrow{d} N\left(\mu, \frac{\sigma^2}{n}\right)$$

In [ ]:
# CLT Demonstration with SPY returns
print("=" * 60)
print("CENTRAL LIMIT THEOREM DEMONSTRATION")
print("=" * 60)

spy_returns = returns['SPY'].values
true_mean = np.mean(spy_returns)
true_std = np.std(spy_returns, ddof=1)

print(f"\nPopulation: SPY Daily Returns (n={len(spy_returns)})")
print(f"True mean: {true_mean:.6f}")
print(f"True std:  {true_std:.6f}")

# Simulate sampling distributions
np.random.seed(42)
sample_sizes = [5, 20, 50, 250]
n_simulations = 10000

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, n in enumerate(sample_sizes):
    # Generate sample means
    sample_means = [np.mean(np.random.choice(spy_returns, n, replace=True)) 
                    for _ in range(n_simulations)]
    
    # Expected std error
    expected_se = true_std / np.sqrt(n)
    
    # Plot
    axes[i].hist(sample_means, bins=50, density=True, alpha=0.7, 
                 color='steelblue', edgecolor='white')
    
    # Normal fit
    x = np.linspace(min(sample_means), max(sample_means), 200)
    axes[i].plot(x, stats.norm.pdf(x, true_mean, expected_se), 'r-', lw=2, 
                 label=f'N(μ, σ/√n)')
    axes[i].axvline(true_mean, color='green', lw=2, linestyle='--', label='True μ')
    
    axes[i].set_xlabel('Sample Mean', fontsize=10)
    axes[i].set_ylabel('Density', fontsize=10)
    axes[i].set_title(f'n = {n}, SE = {expected_se:.6f}', fontsize=11, fontweight='bold')
    axes[i].legend(fontsize=8)
    axes[i].grid(True, alpha=0.3)

plt.suptitle('Central Limit Theorem: Distribution of Sample Means', fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n📊 CLT Implications:")
print("   - Larger samples → narrower distribution → more precise estimates")
print("   - Standard Error = σ/√n")
print("   - Works even though returns have fat tails!")

## 2. Confidence Intervals for Returns

In [ ]:
# Confidence Intervals
print("=" * 60)
print("CONFIDENCE INTERVALS FOR RETURNS")
print("=" * 60)

spy_returns = returns['SPY'].values
n = len(spy_returns)

# Point estimates
mean_daily = np.mean(spy_returns)
std_daily = np.std(spy_returns, ddof=1)
se_mean = std_daily / np.sqrt(n)

# Confidence levels
confidence_levels = [0.90, 0.95, 0.99]

print(f"\nSample: n = {n}")
print(f"Daily mean: {mean_daily:.6f}")
print(f"Standard error: {se_mean:.6f}")
print("\n" + "-" * 60)

for conf in confidence_levels:
    alpha = 1 - conf
    t_critical = stats.t.ppf(1 - alpha/2, df=n-1)
    margin = t_critical * se_mean
    ci_lower = mean_daily - margin
    ci_upper = mean_daily + margin
    
    # Annualize
    ci_lower_ann = ci_lower * 252
    ci_upper_ann = ci_upper * 252
    
    print(f"\n{conf:.0%} Confidence Interval:")
    print(f"  Daily:    [{ci_lower:.6f}, {ci_upper:.6f}]")
    print(f"  Annual:   [{ci_lower_ann:.2%}, {ci_upper_ann:.2%}]")

# Visualize
fig, ax = plt.subplots(figsize=(12, 5))

# Rolling mean with confidence bands
window = 252
rolling_mean = returns['SPY'].rolling(window).mean() * 252
rolling_std = returns['SPY'].rolling(window).std() * np.sqrt(252)
rolling_se = rolling_std / np.sqrt(window)

rolling_mean.plot(ax=ax, color='blue', lw=1.5, label='Rolling Mean (Annualized)')
ax.fill_between(rolling_mean.index, 
                rolling_mean - 1.96 * rolling_se, 
                rolling_mean + 1.96 * rolling_se, 
                alpha=0.3, color='blue', label='95% CI')
ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.set_xlabel('Date', fontsize=11)
ax.set_ylabel('Annualized Return', fontsize=11)
ax.set_title('SPY Rolling Mean Return with 95% Confidence Interval', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Bootstrap Methods - Robust Estimation

In [ ]:
# Bootstrap Confidence Intervals
print("=" * 60)
print("BOOTSTRAP CONFIDENCE INTERVALS")
print("=" * 60)

np.random.seed(42)
spy_returns = returns['SPY'].values
n_bootstrap = 10000

def bootstrap_ci(data, statistic_func, n_bootstrap=10000, confidence=0.95):
    """Calculate bootstrap confidence interval for any statistic."""
    n = len(data)
    bootstrap_stats = []
    
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, n, replace=True)
        bootstrap_stats.append(statistic_func(sample))
    
    alpha = 1 - confidence
    lower = np.percentile(bootstrap_stats, alpha/2 * 100)
    upper = np.percentile(bootstrap_stats, (1 - alpha/2) * 100)
    
    return np.array(bootstrap_stats), lower, upper

# Bootstrap for various statistics
print(f"\nBootstrap Simulation: {n_bootstrap:,} iterations")
print("-" * 60)

# Mean
boot_means, mean_lower, mean_upper = bootstrap_ci(spy_returns, np.mean)
print(f"\nMean Return (daily):")
print(f"  Point estimate: {np.mean(spy_returns):.6f}")
print(f"  95% Bootstrap CI: [{mean_lower:.6f}, {mean_upper:.6f}]")

# Sharpe Ratio
def sharpe_ratio(r, rf=0.05/252):
    return (np.mean(r) - rf) / np.std(r, ddof=1) * np.sqrt(252)

boot_sharpes, sharpe_lower, sharpe_upper = bootstrap_ci(spy_returns, sharpe_ratio)
print(f"\nSharpe Ratio:")
print(f"  Point estimate: {sharpe_ratio(spy_returns):.3f}")
print(f"  95% Bootstrap CI: [{sharpe_lower:.3f}, {sharpe_upper:.3f}]")

# Visualize bootstrap distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Mean bootstrap
axes[0].hist(boot_means * 252 * 100, bins=50, density=True, alpha=0.7, 
             color='steelblue', edgecolor='white')
axes[0].axvline(np.mean(spy_returns) * 252 * 100, color='red', lw=2, label='Point Estimate')
axes[0].axvline(mean_lower * 252 * 100, color='orange', lw=2, linestyle='--', label='95% CI')
axes[0].axvline(mean_upper * 252 * 100, color='orange', lw=2, linestyle='--')
axes[0].set_xlabel('Annualized Return (%)', fontsize=11)
axes[0].set_ylabel('Density', fontsize=11)
axes[0].set_title('Bootstrap Distribution: Mean Return', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Sharpe bootstrap
axes[1].hist(boot_sharpes, bins=50, density=True, alpha=0.7, 
             color='steelblue', edgecolor='white')
axes[1].axvline(sharpe_ratio(spy_returns), color='red', lw=2, label='Point Estimate')
axes[1].axvline(sharpe_lower, color='orange', lw=2, linestyle='--', label='95% CI')
axes[1].axvline(sharpe_upper, color='orange', lw=2, linestyle='--')
axes[1].axvline(0, color='black', lw=1.5, linestyle=':', label='Sharpe = 0')
axes[1].set_xlabel('Sharpe Ratio', fontsize=11)
axes[1].set_ylabel('Density', fontsize=11)
axes[1].set_title('Bootstrap Distribution: Sharpe Ratio', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Bootstrap Advantage: No distributional assumptions!")

## 4. How Many Observations Do We Need?

In [ ]:
# Sample Size Analysis
print("=" * 60)
print("SAMPLE SIZE ANALYSIS: How Long to Detect Signal?")
print("=" * 60)

# Given a true Sharpe ratio, how many years to detect it?
def years_to_detect(true_sharpe, confidence=0.95, power=0.80):
    """Calculate years needed to detect a given Sharpe ratio."""
    z_alpha = stats.norm.ppf(1 - (1 - confidence)/2)
    z_beta = stats.norm.ppf(power)
    
    # Standard error of Sharpe ≈ sqrt((1 + 0.5*SR^2) / n)
    # Solve for n: n = (z_alpha + z_beta)^2 * (1 + 0.5*SR^2) / SR^2
    n_days = ((z_alpha + z_beta)**2 * (1 + 0.5 * true_sharpe**2)) / true_sharpe**2
    n_years = n_days / 252
    
    return n_years

sharpe_values = [0.5, 1.0, 1.5, 2.0, 3.0]
print(f"\nYears needed to detect Sharpe ratio (95% CI, 80% power):")
print("-" * 40)
print(f"{'True Sharpe':<15} {'Years Needed':<15}")
print("-" * 40)
for sr in sharpe_values:
    years = years_to_detect(sr)
    print(f"{sr:<15.1f} {years:<15.1f}")

# Visualize CI width vs sample size
sample_sizes = np.arange(21, 2520, 21)  # 1 month to 10 years
ci_widths = []

for n in sample_sizes:
    se = np.std(spy_returns) / np.sqrt(n)
    width = 2 * 1.96 * se * 252 * 100  # Annualized percentage
    ci_widths.append(width)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(sample_sizes / 252, ci_widths, 'b-', lw=2)
ax.axhline(10, color='red', linestyle='--', label='±10% target')
ax.axhline(5, color='green', linestyle='--', label='±5% target')
ax.set_xlabel('Years of Data', fontsize=11)
ax.set_ylabel('95% CI Width (%)', fontsize=11)
ax.set_title('Confidence Interval Width vs Sample Size', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 10)
plt.tight_layout()
plt.show()

print("\n⚠️ Key Insight: Need ~4 years of daily data for reasonably precise estimates!")

## 📝 Key Takeaways - Day 5

### Central Limit Theorem:
- Sample means → Normal as n increases
- Standard Error = σ / √n
- Works even for non-Normal returns

### Confidence Intervals:
- Parametric: Assume underlying distribution
- Bootstrap: No distributional assumptions
- Wider CI = more uncertainty

### Sample Size Rules of Thumb:
- Sharpe 0.5 → ~16 years to detect
- Sharpe 1.0 → ~4 years to detect
- Sharpe 2.0 → ~1 year to detect

### Interview Questions:
- "How long would you backtest a strategy?"
- "What's the standard error of the Sharpe ratio?"
- "When would you use bootstrap vs parametric CI?"
- "How does sample size affect estimation precision?"